In [8]:
!pip install transformers

     |████████████████████████████████| 552kB 8.3MB/s 
     |████████████████████████████████| 1.0MB 21.4MB/s 
     |████████████████████████████████| 870kB 59.7MB/s 
     |████████████████████████████████| 3.7MB 54.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=9fe2ddcbc2aad8c82e427508959e031737ce537a8266c37d8b8b5b3746256f8d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [3]:
import torch
from transformers import *

# Transformers has a unified API
# for 10 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [
          (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
         ]


# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples
    break

In [6]:
last_hidden_states.shape

torch.Size([1, 9, 768])

In [21]:
aa.squeeze(0).mean(axis=0).shape

(768,)

In [1]:
!pip install wget
import wget
import os
print('Downloading dataset')# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
  wget.download(url, './cola_public_1.1.zip')

ERROR! Session/line number was not unique in database. History logging moved to new session 60
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=64ddd42e975527f76d5c2c1d7c3adf5e34dac36b1791ae4ce9cf14a97ce30698
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
if not os.path.exists('./cola_public/'):
  !unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [4]:
import pandas as pd# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
3747,ks08,1,NaN,The kids in our class have arrived safely.
3076,l-93,0,*,Ellen conferred to Helen.
352,bc01,0,*,John kisses often Mary.
1497,r-67,1,NaN,I promised that tomorrow he would be there.
5916,c_13,1,NaN,Molly gave Calvin a kiss.
5112,ks08,0,*,The writer is that gets you so involved.
751,bc01,0,*,They represented to the dean Mary as a genuine...
380,bc01,1,NaN,No candidate can predict how many people will ...
5976,c_13,1,NaN,Calvin has a peanut.
7315,sks13,0,*,I said.


In [0]:
sentences = df.sentence.values
labels = df.label.values

In [9]:
from transformers import BertTokenizer# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [10]:
df.sentence.iloc[0]

"Our friends won't buy this analysis, let alone the next one we propose."

In [0]:
import torch
input_ids = torch.tensor([tokenizer.encode(df.sentence.iloc[0], add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.

In [21]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []# For every sentence...
for sent in sentences:
 # `encode` will:
 # (1) Tokenize the sentence.
 # (2) Prepend the `[CLS]` token to the start.
 # (3) Append the `[SEP]` token to the end.
 # (4) Map tokens to their IDs.
 encoded_sent = tokenizer.encode(
 sent, # Sentence to encode.
 add_special_tokens = True, # Add '[CLS]' and '[SEP]' # This function also supports truncation and conversion
 # to pytorch tensors, but we need to do padding, so we
 # can't use these features :( .
 #max_length = 128, # Truncate all sentences.
 #return_tensors = 'pt', # Return pytorch tensors.
 )

 # Add the encoded sentence to the list.
 input_ids.append(encoded_sent)# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences# Set the maximum sequence length.
# I've chosen 64 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length of 47...
MAX_LEN = 64
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
 value=0, truncating="post", padding="post")
print('\Done.')

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]

Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0
\Done.


In [0]:
attention_masks = []
for sent in input_ids:

  # Create the attention mask.
  # - If a token ID is 0, then it's padding, set the mask to 0.
  # - If a token ID is > 0, then it's a real token, set the mask to 1.
  att_mask = [int(token_id > 0) for token_id in sent]

  # Store the attention mask for this sentence.
  attention_masks.append(att_mask)

In [27]:
input_ids.shape, labels.shape, len(attention_masks), labels.shape

((8551, 64), (8551,), 8551, (8551,))

In [0]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
 random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
 random_state=2018, test_size=0.1)

In [29]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

ERROR! Session/line number was not unique in database. History logging moved to new session 65


In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.
batch_size = 32# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [32]:
from transformers import BertForSequenceClassification, AdamW, BertConfig# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
 "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
 num_labels = 2, # The number of output labels--2 for binary classification.
 # You can increase this for multi-class tasks. 
 output_attentions = False, # Whether the model returns attentions weights.
 output_hidden_states = False, # Whether the model returns all hidden-states.
)# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [34]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
  print('\n==== First Transformer ====\n')
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
  print('\n==== Output Layer ====\n')
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)

==== First Transformer ====

bert.embeddings.position_embeddings.weight                (512, 768)

==== First Transformer ====

bert.embeddings.token_type_embeddings.weight                (2, 768)

==== First Transformer ====

bert.embeddings.LayerNorm.weight                              (768,)

==== First Transformer ====

bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)

==== Output Layer ====

bert.encoder.layer.0.attention.self.query.bias                (768,)

==== Output Layer ====

bert.encoder.layer.0.attention.self.key.weight            (768, 768)

==== Output Layer ====

bert.encoder.layer.0.attention.self.key.bias                  (768,)

==== Output Layer ====

bert.encoder.layer.0.attention.self.value.weight  

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
 lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
 eps = 1e-8 # args.adam_epsilon - default is 1e-8.
 )
from transformers import get_linear_schedule_with_warmup# Number of training epochs (authors recommend between 2 and 4)
epochs = 4# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
 num_warmup_steps = 0, # Default value in run_glue.py
 num_training_steps = total_steps)

In [0]:
import numpy as np# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
 pred_flat = np.argmax(preds, axis=1).flatten()
 labels_flat = labels.flatten()
 return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime
def format_time(elapsed):
 '''
 Takes a time in seconds and returns a string hh:mm:ss
 '''
 # Round to the nearest second.
 elapsed_rounded = int(round((elapsed)))

 # Format as hh:mm:ss
 return str(datetime.timedelta(seconds=elapsed_rounded))

In [43]:
import torch# If there's a GPU available...
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
 device = torch.device("cuda")
 print('There are %d GPU(s) available.' % torch.cuda.device_count())
 print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [86]:
label_ids

array([1, 0, 1, 1])

In [44]:
import random# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)# Store the average loss after each epoch so we can plot them.
loss_values = []# For each epoch...
for epoch_i in range(0, epochs):

 # ========================================
 # Training
 # ========================================

 # Perform one full pass over the training set. print("")
 print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
 print('Training...') # Measure how long the training epoch takes.
 t0 = time.time() # Reset the total loss for this epoch.
 total_loss = 0 # Put the model into training mode. Don't be mislead--the call to 
 # `train` just changes the *mode*, it doesn't *perform* the training.
 # `dropout` and `batchnorm` layers behave differently during training
 # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
 model.train() # For each batch of training data...
 for step, batch in enumerate(train_dataloader): # Progress update every 40 batches.
  if step % 40 == 0 and not step == 0:
  # Calculate elapsed time in minutes.
    elapsed = format_time(time.time() - t0)

    # Report progress.
    print(' Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed)) # Unpack this training batch from our dataloader. 
  #
  # As we unpack the batch, we'll also copy each tensor to the GPU using the 
  # `to` method.
  #
  # `batch` contains three pytorch tensors:
  # [0]: input ids 
  # [1]: attention masks
  # [2]: labels 
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device) # Always clear any previously calculated gradients before performing a
  # backward pass. PyTorch doesn't do this automatically because 
  # accumulating the gradients is "convenient while training RNNs". 
  # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
  model.zero_grad()  # Perform a forward pass (evaluate the model on this training batch).
  # This will return the loss (rather than the model output) because we
  # have provided the `labels`.
  # The documentation for this `model` function is here: 
  # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
  outputs = model(b_input_ids, 
  token_type_ids=None, 
  attention_mask=b_input_mask, 
  labels=b_labels)

  # The call to `model` always returns a tuple, so we need to pull the 
  # loss value out of the tuple.
  loss = outputs[0] # Accumulate the training loss over all of the batches so that we can
  # calculate the average loss at the end. `loss` is a Tensor containing a
  # single value; the `.item()` function just returns the Python value 
  # from the tensor.
  total_loss += loss.item() # Perform a backward pass to calculate the gradients.
  loss.backward() # Clip the norm of the gradients to 1.0.
  # This is to help prevent the "exploding gradients" problem.
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Update parameters and take a step using the computed gradient.
  # The optimizer dictates the "update rule"--how the parameters are
  # modified based on their gradients, the learning rate, etc.
  optimizer.step() # Update the learning rate.
  scheduler.step() # Calculate the average loss over the training data.
  avg_train_loss = total_loss / len(train_dataloader) 

  # Store the loss value for plotting the learning curve.
  loss_values.append(avg_train_loss)
  print("")
  print(" Average training loss: {0:.2f}".format(avg_train_loss))
  print(" Training epcoh took: {:}".format(format_time(time.time() - t0)))

 # ========================================
 # Validation
 # ========================================
 # After the completion of each training epoch, measure our performance on
 # our validation set. print("")
 print("Running Validation...") 
 t0 = time.time() # Put the model in evaluation mode--the dropout layers behave differently
 # during evaluation.
 model.eval() # Tracking variables 
 eval_loss, eval_accuracy = 0, 0
 nb_eval_steps, nb_eval_examples = 0, 0 # Evaluate data for one epoch
 for batch in validation_dataloader:

  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up validation
  with torch.no_grad():  # Forward pass, calculate logit predictions.
    # This will return the logits rather than the loss because we have
    # not provided labels.
    # token_type_ids is the same as the "segment ids", which 
    # differentiates sentence 1 and 2 in 2-sentence tasks.
    # The documentation for this `model` function is here: 
    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    outputs = model(b_input_ids, 
    token_type_ids=None, 
    attention_mask=b_input_mask)

    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0] # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy # Track the number of batches
    nb_eval_steps += 1 # Report the final accuracy for this validation run.
    print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print(" Validation took: {:}".format(format_time(time.time() - t0)))
    print("")
print("Training complete!")

======== Epoch 1 / 4 ========
Training...

 Average training loss: 0.00
 Training epcoh took: 0:00:01

 Average training loss: 0.01
 Training epcoh took: 0:00:01

 Average training loss: 0.01
 Training epcoh took: 0:00:01

 Average training loss: 0.01
 Training epcoh took: 0:00:02

 Average training loss: 0.01
 Training epcoh took: 0:00:02

 Average training loss: 0.02
 Training epcoh took: 0:00:02

 Average training loss: 0.02
 Training epcoh took: 0:00:03

 Average training loss: 0.02
 Training epcoh took: 0:00:03

 Average training loss: 0.02
 Training epcoh took: 0:00:04

 Average training loss: 0.03
 Training epcoh took: 0:00:04

 Average training loss: 0.03
 Training epcoh took: 0:00:04

 Average training loss: 0.03
 Training epcoh took: 0:00:05

 Average training loss: 0.03
 Training epcoh took: 0:00:05

 Average training loss: 0.04
 Training epcoh took: 0:00:05

 Average training loss: 0.04
 Training epcoh took: 0:00:06

 Average training loss: 0.04
 Training epcoh took: 0:00:0

In [45]:
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

In [46]:
import pandas as pd
# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))
# Create sentence and label lists
sentences = df.sentence.values
labels = df.label.values
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 516



In [47]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')

Predicting labels for 516 test sentences...
DONE.


In [48]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

Positive samples: 354 of 516 (68.60%)


In [81]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []
# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')
# For each input batch...
for i in range(len(true_labels)):
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  try:
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)
  except:
    pass

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning:

invalid value encountered in double_scalars



In [83]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]
# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
print('MCC: %.3f' % mcc)

MCC: 0.561


In [89]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py

--2020-04-01 13:59:33--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34311 (34K) [text/plain]
Saving to: ‘run_language_modeling.py’

run_language_modeli 100%[===================>]  33.51K  --.-KB/s    in 0.005s  

2020-04-01 13:59:33 (7.20 MB/s) - ‘run_language_modeling.py’ saved [34311/34311]



In [94]:
!wget -nc -O /content/presidential_speeches_test.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
!wget -nc -O /content/presidential_speeches_valid.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_valid.txt
!wget -nc -O /content/presidential_speeches_train.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_train.txt

--2020-04-01 14:09:08--  https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622537 (608K) [text/plain]
Saving to: ‘/content/presidential_speeches_test.txt’

/content/presidenti 100%[===================>] 607.95K  --.-KB/s    in 0.02s   

2020-04-01 14:09:08 (25.8 MB/s) - ‘/content/presidential_speeches_test.txt’ saved [622537/622537]

--2020-04-01 14:09:11--  https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.13

In [95]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/presidential_speeches' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/presidential_speeches_train.txt \
    --do_eval \
    --eval_data_file=/content/presidential_speeches_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5


Streaming output truncated to the last 5000 lines.

Evaluating:  36% 312/871 [00:22<00:40, 13.82it/s]

Evaluating:  36% 314/871 [00:22<00:40, 13.85it/s]

Evaluating:  36% 316/871 [00:22<00:39, 13.92it/s]

Evaluating:  37% 318/871 [00:22<00:39, 13.84it/s]

Evaluating:  37% 320/871 [00:23<00:40, 13.72it/s]

Evaluating:  37% 322/871 [00:23<00:39, 13.82it/s]

Evaluating:  37% 324/871 [00:23<00:39, 13.80it/s]

Evaluating:  37% 326/871 [00:23<00:39, 13.81it/s]

Evaluating:  38% 328/871 [00:23<00:39, 13.85it/s]

Evaluating:  38% 330/871 [00:23<00:38, 13.87it/s]

Evaluating:  38% 332/871 [00:24<00:39, 13.72it/s]

Evaluating:  38% 334/871 [00:24<00:38, 13.80it/s]

Evaluating:  39% 336/871 [00:24<00:38, 13.78it/s]

Evaluating:  39% 338/871 [00:24<00:38, 13.76it/s]

Evaluating:  39% 340/871 [00:24<00:38, 13.89it/s]

Evaluating:  39% 342/871 [00:24<00:37, 13.95it/s]

Evaluating:  39% 344/871 [00:24<00:37, 13.93it/s]

Evaluating:  40% 346/871 [00:25<00:37, 13.84it/s]

Evaluating:  40% 348/871 [00:2

In [96]:
!ls '/content/drive/My Drive/finetuned_models/presidential_speeches'


checkpoint-1000  checkpoint-3000   pytorch_model.bin	    vocab.json
checkpoint-1500  config.json	   special_tokens_map.json
checkpoint-2000  eval_results.txt  tokenizer_config.json
checkpoint-2500  merges.txt	   training_args.bin


In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [98]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/presidential_speeches/checkpoint-1500'

# You should try out other prompts as well as no prompt at all.
PROMPT = '<title=\"Remarks on Mission to Mars\">'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=100,  # Number of tokens to generate.
  num_return_sequences=3,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

Running on device:  cuda


NameError: ignored